In [ ]:
#Sources 

# https://en.wikipedia.org/wiki/Rotation_matrix
# https://docs.opencv.org/3.4/d4/d94/tutorial_camera_calibration.html
# 

import numpy as np
import math as m
import matplotlib.pyplot as plt
import os
import cv2 

np.set_printoptions(suppress=True,
                    formatter={'float_kind': '{:f},'.format})

In [ ]:
#Resolution

opencv_mtx = np.loadtxt(fname= os.getcwd() + '/storage/opencv_matrix.txt', delimiter=',')
image_info =  np.loadtxt(fname= os.getcwd() +  '/storage/image_info.txt', delimiter=',')
w = image_info[0]
h = image_info[1]

image_points = [
    np.array([0, 0]),
    np.array([w, 0]),
    np.array([w, h]),
    np.array([0, h]),
    np.array([0, 0]),
]

image_points_scaled = [
    np.array([-w, -h]),
    np.array([w, -h]),
    np.array([w, h]),
    np.array([-w, h]),
    np.array([-w, -h]),
]

def get_rot_mtx(x,y,z):
    return np.array([
        [m.cos(z)*m.cos(y)  ,m.cos(z)*m.sin(y)*m.sin(x) - m.sin(z)*m.cos(x)   ,m.cos(z)*m.sin(y)*m.cos(x) + m.sin(z)*m.sin(x) , 0],
        [m.sin(z)*m.cos(y)  ,m.sin(z)*m.sin(y)*m.sin(x) + m.cos(z)*m.cos(x)   ,m.sin(z)*m.sin(y)*m.cos(x) - m.cos(z)*m.sin(x) , 0],
        [-m.sin(y)          ,m.cos(y)*m.sin(x)                                ,m.cos(y)*m.cos(x)                              , 0],
    ])



def get_world_points(delta_x, delta_y, dis, size):
    return [
        np.array([-size + delta_x, -size + delta_y, dis  ,0]),  #bottom left
        np.array([ size + delta_x, -size + delta_y, dis  ,0]),  #bottom right
        np.array([ size + delta_x,  size + delta_y, dis  ,0]),  #top right 
        np.array([-size + delta_x,  size + delta_y, dis  ,0]),  #top left
    ]

In [ ]:


size = 32.5

xpos = 0
ypos = 0
zpos = 0

xrot = 0   #degrees
yrot = 0    #degrees

xrad = xrot * (m.pi/180)

yrad = yrot * (m.pi/180)

print()
# cx = 80/m.cos(xrad)
# cy = 80/m.cos(yrad)

# ax = m.sqrt(m.pow(cx,2)-m.pow(0.2,2))
# ay = m.sqrt(m.pow(cy,2)-m.pow(0.2,2))

# if(xrot < 0):
#     ax = -ax
# elif(xrot==0):
#     ax =0

# if(yrot < 0):
#     ay = -ay
# elif(yrot==0):
#     ay =0

world_points = get_world_points(delta_x=xpos,delta_y=ypos,dis=zpos,size=size)
rotation_mtx = get_rot_mtx(xrad,yrad,0)

point_set = []

for p in world_points:
    p_rotated = np.dot(rotation_mtx, p)
    screen_point = opencv_mtx.dot(p_rotated)/p_rotated[2]
    # screen_point, _ = cv2.projectPoints(np.array([p_rotated]), np.zeros(3), np.zeros(3), opencv_mtx, np.zeros(5))
    point_set.append(np.array([screen_point[0], screen_point[1]]))
    
point_set.append(point_set[0])

xs, ys = zip(*image_points) #create lists of x and y values
xs1, ys1 = zip(*image_points_scaled) #create lists of x and y values
scale = 150

plt.figure(figsize=(w/scale,h/scale))
plt.plot(xs,ys)
# plt.plot(xs1,ys1)


xs1, ys1 = zip(*point_set)
plt.plot(xs1*10,ys1*10)  

plt.show() # if you need...


In [ ]:
position = ['bottom left','bottom right','top right','top left']

for x in range(0, 4):
    p = point_set[x]
    print(p, '  ', position[x] , '   ', x)
    
size_top = size_bot = point_set[3] - point_set[2]
print('top ',np.sqrt(size_top.dot(size_top)))

size_bot = point_set[0] - point_set[1]
print('bot ', np.sqrt(size_bot.dot(size_bot)))

size_left = point_set[0] - point_set[3]
print('left ', np.sqrt(size_left.dot(size_left)))

size_right = point_set[1] - point_set[2]
print('right ', np.sqrt(size_right.dot(size_right)))
